In [1]:
import spacy

#### Task 1

за допомогою класу Matcher знайти і видалити такі помилки, як зайві пробіли та знаки пунктуації

In [2]:
text_file = open("text2.txt", "r")
text1 = text_file.read()
text_file.close()

In [3]:
nlp = spacy.load('en_core_web_sm')

In [4]:
text1

"It was a 18:15 when we left Baker Street 221b, and it still\nwanted ten minutes to the hour 6-50pm when we found ourselves in Serpentine\nAvenue@mail.com. It was already dusk,, and the lamps were just being lighted as\nwe paced up and down in front of Briony Lodge, waiting for the coming\nof its (343)-45-23 occupant. The house 345-A was just such as I had pictured it from\nSherlock  Holmes's succinct,,,, description, but the locality appeared to\nbe less private that I    expected 456-45-34. On the contrary, for a small Street 23\nin a quiet neighborhood here1232@emm.uk, it was remarkably animated. There was a\ngroup of   shabbily-dressed men smoking and laughing in a corner, a\nscissors-grinder +308-45-32-21 with his wheel, two guardsmen who were flirting with a\nnurse-girl, and several well-dressed young men who were lounging up and\ndown with cigars in their mouths j.watson@gmail.com!!!"

In [5]:
from spacy.matcher import Matcher

matcher = Matcher(nlp.vocab)
pattern_extra_spaces = [{"IS_SPACE": True, "OP": "+"}]
pattern_repeat_chars = [{"IS_PUNCT": True, "OP": "{2,}"}]

matcher.add("EXTRA_SPACES", [pattern_extra_spaces])
matcher.add("REPEAT_CHARS", [pattern_repeat_chars])

In [6]:
def search_mistakes(txt_part):
    text_nlp = nlp(txt_part)
    matches = matcher(text_nlp)
    new_tokens = []
    punctuation_count = 0

    for token in text_nlp:
        if any(start <= token.i < end for _, start, end in matches):
            if token.is_punct:
                if punctuation_count == 0:
                    new_tokens.append(token)
                    punctuation_count += 1
            else:
                new_tokens.append(token)
        else:
            new_tokens.append(token)

    clean_tokens = []

    for i, token in enumerate(new_tokens):
        if not token.is_space:
            if not (token.is_punct and i > 0 and new_tokens[i - 1].is_punct):
                clean_tokens.append(token)

    clean_text = " ".join([token.text for token in clean_tokens])
    return clean_text

In [7]:
text1 = search_mistakes(text1)
print(text1)

It was a 18:15 when we left Baker Street 221b , and it still wanted ten minutes to the hour 6 - 50pm when we found ourselves in Serpentine Avenue@mail.com . It was already dusk , and the lamps were just being lighted as we paced up and down in front of Briony Lodge , waiting for the coming of its ( 343)-45 - 23 occupant . The house 345 - A was just such as I had pictured it from Sherlock Holmes 's succinct description , but the locality appeared to be less private that I expected 456 - 45 - 34 . On the contrary , for a small Street 23 in a quiet neighborhood here1232@emm.uk , it was remarkably animated . There was a group of shabbily - dressed men smoking and laughing in a corner , a scissors - grinder +308 - 45 - 32 - 21 with his wheel , two guardsmen who were flirting with a nurse - girl , and several well - dressed young men who were lounging up and down with cigars in their mouths j.watson@gmail.com


#### Task 2

In [8]:
text_file = open("lab7-3.txt", "r")
text2 = text_file.read()
text_file.close()

In [9]:
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
import string
import re

punct_list = [char for char in string.punctuation]
stop_list = stopwords.words('english') + punct_list


def clear_punc(txt_arr):
    new_txt_arr = []
    for txt_part in txt_arr:
        new_txt = re.sub(r'[^\w\s]', '', txt_part)
        if new_txt != "":
            new_txt_arr.append(new_txt)
    return new_txt_arr


def clean_text_in_sentence(sentence):
    clean_list = [word for word in sentence.lower().split() if word not in stop_list]
    return clear_punc(clean_list)


def clean_text(text_sample):
    new_text_arr = []
    for sentence in sent_tokenize(text_sample):
        sentence = clean_text_in_sentence(sentence)
        for word in sentence:
            new_text_arr.append(word)
    return new_text_arr

In [10]:
text2 = ' '.join(clean_text(text2))

а) Вивести леми до слів у тексті.

In [11]:
text_nlp = nlp(text2)

for token in text_nlp:
    print(token.text, token.lemma_)

gordon gordon
brown brown
issued issue
rallying rally
cry cry
telling tell
supporters supporter
stakes stake
high high
stay stay
home home
protest protest
vote vote
forthcoming forthcome
general general
election election
chancellor chancellor
said say
vote vote
expected expect
fall fall
5 5
may may
give give
clear clear
fundamental fundamental
choice choice
labour labour
investment investment
conservative conservative
cuts cut
speaking speak
labours labour
spring spring
conference conference
gateshead gateshead
mr mr
brown brown
claimed claim
nhs nhs
safe safe
conservative conservative
hands hand
said say
tory tory
plans plan
cut cut
рр35bn рр35bn
tax tax
would would
cut cut
deep deep
public public
service service
packed pack
audience audience
gatesheads gateshead
sage sage
centre centre
chancellor chancellor
said say
cuts cut
proposed propose
shadow shadow
chancellor chancellor
oliver oliver
letwin letwin
nasa nasa
equivalent equivalent
sacking sack
every every
teacher teacher
gp gp
n

б) Знайти та вивести всі прикметники, які присутні у тексті.

In [12]:
adjectives = [token.text for token in text_nlp if token.pos_ == "ADJ"]
print(adjectives)

['high', 'general', 'clear', 'fundamental', 'conservative', 'safe', 'conservative', 'deep', 'public', 'meta', 'negative', 'unemployed', 'central', 'conservative', 'deep', 'public', 'proud', 'more', 'economic', 'third', 'warm', 'teenage', 'next', 'poorest', 'national', 'interactive', 'public']


в) Знайти та вивести організації та дати, які присутні у тексті.

In [13]:
organizations = [entity.text for entity in text_nlp.ents if entity.label_ == "ORG"]
dates = [entity.text for entity in text_nlp.ents if entity.label_ == "DATE"]

print("Організації:", organizations)
print("Дати:", dates)

Організації: ['nasa', 'google inc government', 'pepsi', 'рр35bn']
Дати: ['fall 5', '18 june', '2003 250000', 'seven years', 'next five years']


In [14]:
ner_tagged = [(word.text, word.ent_type_) for word in text_nlp]
named_entities = []
temp_entity_name = ""
temp_named_entity = None
for term, tag in ner_tagged:
    if tag:
        temp_entity_name = ' '.join([temp_entity_name, term]).strip()
        temp_named_entity = (temp_entity_name, tag)
    else:
        if temp_named_entity:
            named_entities.append(temp_named_entity)
            temp_entity_name = ""
            temp_named_entity = None
print(named_entities)

[('gordon brown', 'PERSON'), ('fall 5', 'DATE'), ('рр35bn', 'GPE'), ('oliver letwin nasa', 'ORG'), ('google inc government', 'ORG'), ('britain', 'GPE'), ('pepsi', 'ORG'), ('10', 'CARDINAL'), ('15', 'CARDINAL'), ('18 june', 'DATE'), ('2003 250000', 'DATE'), ('one million', 'CARDINAL'), ('three million', 'CARDINAL'), ('tory', 'NORP'), ('britain', 'GPE'), ('рр35bn', 'ORG'), ('britain', 'GPE'), ('2008', 'CARDINAL'), ('third', 'ORDINAL'), ('seven years', 'DATE'), ('brown', 'PERSON'), ('next five years', 'DATE'), ('100', 'CARDINAL'), ('16 17yearolds', 'CARDINAL'), ('later saturday', 'TIME')]


In [15]:
from spacy import displacy
displacy.render(text_nlp, style='ent', jupyter=True)